In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


In [18]:
print(logits)
print(pred_probab)

tensor([[ 0.0258,  0.0368,  0.0429,  0.0138,  0.0502, -0.0541, -0.0479, -0.0087,
         -0.0226, -0.0430]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.1026, 0.1037, 0.1044, 0.1014, 0.1051, 0.0947, 0.0953, 0.0991, 0.0978,
         0.0958]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [19]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [20]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [21]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [22]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1731, -0.3854,  0.2743, -0.0997,  0.1505, -0.0799,  0.1434, -0.2275,
          0.1288, -0.2327, -0.3589,  0.0547,  0.2129,  0.2084, -0.8807,  0.1476,
         -0.1357, -0.0485,  0.2850,  0.1323],
        [ 0.4149, -0.3634,  0.2724, -0.0595,  0.2189,  0.1446,  0.0908, -0.2583,
          0.0335,  0.0192, -0.2355,  0.0460, -0.0842,  0.2778, -0.7797, -0.2621,
         -0.0718,  0.1292,  0.4060, -0.1405],
        [ 0.3091, -0.1469,  0.5212, -0.0835,  0.6479,  0.0396, -0.0166,  0.0281,
         -0.0569, -0.1171,  0.2367,  0.1478,  0.1018, -0.1579, -0.6675,  0.0119,
          0.1726,  0.3489,  0.2282,  0.2163]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1731, 0.0000, 0.2743, 0.0000, 0.1505, 0.0000, 0.1434, 0.0000, 0.1288,
         0.0000, 0.0000, 0.0547, 0.2129, 0.2084, 0.0000, 0.1476, 0.0000, 0.0000,
         0.2850, 0.1323],
        [0.4149, 0.0000, 0.2724, 0.0000, 0.2189, 0.1446, 0.0908, 0.0000, 0.0335,
         0.0192, 0.0000, 0.0460, 0.0000, 0.2778, 0.00

In [24]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [25]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [26]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0331, -0.0200,  0.0212,  ..., -0.0270, -0.0019, -0.0071],
        [-0.0333, -0.0228, -0.0175,  ..., -0.0322, -0.0081, -0.0080]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0035, -0.0271], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0059, -0.0400,  0.0024,  ...,  0.0401,  0.0255, -0.0304],
        [ 0.0143, -0.0048, -0.0103,  ..., -0.0401, -0.0080, -0.0005]],
       device='cuda:0', grad_fn=<Sl